In [1]:
import os
root_dir = "/hpcwork/ui556004/results/musicbert_hf/test/salamied"
input_files = os.listdir(root_dir)
input_files = [os.path.join(root_dir, file) for file in input_files]
input_files

['/hpcwork/ui556004/results/musicbert_hf/test/salamied/quality.csv',
 '/hpcwork/ui556004/results/musicbert_hf/test/salamied/inversion.csv',
 '/hpcwork/ui556004/results/musicbert_hf/test/salamied/key_pc_mode.csv',
 '/hpcwork/ui556004/results/musicbert_hf/test/salamied/primary_alteration_primary_degree_secondary_alteration_secondary_degree.csv']

In [2]:
from transformers import AutoConfig

config = AutoConfig.from_pretrained("/hpcwork/ui556004/results/nas_layers_extended_new/trial_0058/checkpoint-34000")
id2label_map = config.multitask_id2label

/home/ui556004/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import ast
import numpy as np
import sklearn
import pandas as pd
import sklearn.metrics
df = pd.read_csv(input_files[0])
paths_df_indices = df[["path", "indices"]]
df = df.drop(["indices"], axis=1) #  
for i, input_file in enumerate(input_files[1:], start=1):
    new_df = pd.read_csv(input_file).drop(["path", "indices"], axis=1)
    df = df.merge(
        new_df,
        left_index=True,
        right_index=True,
        suffixes=("", f"_{i}"),
        how="outer",
    )
df = df.rename({"predicted": "predicted_0", "labels": "labels_0"}, axis=1)


In [4]:
id2label_map

{'inversion': {'-100': '<pad>',
  '0': '<unk>',
  '2': '<s>',
  '3': '</s>',
  '4': '0.0',
  '5': '1.0',
  '6': '2.0',
  '7': '3.0'},
 'key_pc_mode': {'-100': '<pad>',
  '0': '<unk>',
  '2': '<s>',
  '3': '</s>',
  '4': '0.0M',
  '5': '1.0M',
  '6': '10.0M',
  '7': '11.0M',
  '8': '2.0M',
  '9': '3.0M',
  '10': '4.0M',
  '11': '5.0M',
  '12': '6.0M',
  '13': '7.0M',
  '14': '8.0M',
  '15': '9.0M',
  '16': '0.0m',
  '17': '1.0m',
  '18': '10.0m',
  '19': '11.0m',
  '20': '2.0m',
  '21': '3.0m',
  '22': '4.0m',
  '23': '5.0m',
  '24': '6.0m',
  '25': '7.0m',
  '26': '8.0m',
  '27': '9.0m'},
 'primary_alteration_primary_degree_secondary_alteration_secondary_degree': {'-100': '<pad>',
  '0': '<unk>',
  '2': '<s>',
  '3': '</s>',
  '4': '_I_I',
  '5': '_V_I',
  '6': '_IV_I',
  '7': '_II_I',
  '8': '_VI_I',
  '9': '_VII_I',
  '10': '_V_V',
  '11': '_V_IV',
  '12': '_VII_V',
  '13': '_III_I',
  '14': '#VII_I',
  '15': '_V_II',
  '16': '_V_VI',
  '17': '_x_I',
  '18': '_V_III',
  '19': 'bII_I'

labels_0 : quality \
labels_1: inversion \
labels_2: key_pc_mode \
labels_3: primary_alteration_primary_degree_secondary_alteration_secondary_degree

In [ ]:
def parse_cell(x):
    if not isinstance(x, str):
        return x
    res = ast.literal_eval(x)
    #print(max(res))
    return(max(res))
columms_to_convert = [col for col in df.columns if ("labels" in col )]
columms_to_convert
df[columms_to_convert] = df[columms_to_convert].applymap(parse_cell)
df
df[columms_to_convert].max()

labels_0     14
labels_1      7
labels_2     27
labels_3    185
dtype: int64

In [41]:
def parse_cell(x):
    if not isinstance(x, str):
        return x
    res = ast.literal_eval(x)
    #print(max(res))
    return(max(res))
columms_to_convert = [col for col in df.columns if ("predicted" in col )]
columms_to_convert
df[columms_to_convert] = df[columms_to_convert].applymap(parse_cell)
df
df[columms_to_convert].max() +4

predicted_0     14
predicted_1      7
predicted_2     27
predicted_3    100
dtype: int64

In [31]:
random.choice (list(id2label_map['primary_alteration_primary_degree_secondary_alteration_secondary_degree'].keys())[4:])

'102'

In [37]:
from typing import Dict, List, Union
import random
def get_random_key(id2label_dict):
    # do not choose special symbols 
    keys = list(id2label_dict.keys())[4:]
    random_key =  random.choice(keys)
    return id2label_dict[random_key]
def _decode_ids_to_tokens(ids: np.ndarray, id2label: Dict[int, str]) -> List[str]:
    # Map each int id -> label string; unknown ints -> str(id)
    # in case there is some wrong key just choose the value randomly
    return [id2label.get(str(i), get_random_key(id2label)) for i in ids.tolist()]

In [40]:
from collections import Counter
total_len = 0
mapping_dict = {"labels_0": "quality", "labels_1": "inversion", "labels_2": "key_pc_mode", "labels_3": "primary_alteration_primary_degree_secondary_alteration_secondary_degree"}
decoded_labels_column = []
decoded_predictions_column = []
all_label_values = []
all_predicted_values = []
for ind, row in df.iterrows():
    y_trues = [
        np.array(ast.literal_eval(row[f"labels_{i}"])) 
        for i in range(len(input_files))
    ]
    y_trues_decoded = [_decode_ids_to_tokens(y_true_enc, id2label_map[mapping_dict[f"labels_{i}"]])  for i, y_true_enc in enumerate(y_trues)]
    
    y_preds = [
        np.array(ast.literal_eval(row[f"predicted_{i}"]))
        for i in range(len(input_files))
    ]
    y_preds_decoded = [_decode_ids_to_tokens(y_pred_enc +4, id2label_map[mapping_dict[f"labels_{i}"]])  for i, y_pred_enc in enumerate(y_preds)]
    y_true = ["_".join(str(x) for x in xs) for xs in zip(*y_trues)]

    y_true_decoded = ["_".join(x for x  in xs) for xs in zip(*y_trues_decoded)]
    y_pred_decoded = ["_".join(x for x  in xs) for xs in zip(*y_preds_decoded)]
   # print(y_trues_decoded)
    decoded_labels_column.append(y_true_decoded)
    decoded_predictions_column.append(y_pred_decoded)
    #df.at[ind, "labels_decoded"] = y_true_decoded
    y_pred = ["_".join(str(x + 4) for x in xs) for xs in zip(*y_preds)] 
    uniform_step_col = row["uniform_steps"]

    repeats =  np.array(ast.literal_eval(uniform_step_col))
    y_true = np.repeat(y_true, repeats)
    y_pred = np.repeat(y_pred, repeats)
    #print(Counter(y_true_decoded))
    all_label_values += y_true_decoded
    all_predicted_values += y_pred_decoded
    #all_values.update(set(y_true_decoded))
    assert len(y_true) == len(y_pred)

    unique_labels = sorted(set(y_true) | set(y_pred))

    # precision, recall, f1, support = sklearn.metrics.precision_recall_fscore_support(
    #     y_true, y_pred, average="weighted"
    # )
    total_len += len(y_true)
    accuracy = sklearn.metrics.accuracy_score(y_true, y_pred)#*len(y_true)  
    df.at[ind, "accuracy"] = accuracy
#df["accuracy"].sum() / total_len   # getting an actual accuracy
df["labels_decoded"] = decoded_labels_column
df["preds_decoded"] = decoded_predictions_column

In [7]:
def split_rn(rn):
    rn_wo_degree, degree = rn.split("__")
    quality, inversion, key_pc_mode = rn_wo_degree.split("_")
    return quality, inversion, key_pc_mode, degree

In [8]:
value = 'o7_0.0_0.0m__VII_IV'
split_rn(value)

('o7', '0.0', '0.0m', 'VII_IV')

In [42]:
len(all_predicted_values)

67592

In [20]:
'm_1.0_5.0M__III_I'.split("__")[1]

'III_I'

In [109]:
df.head()

,path,uniform_steps,labels_0,predicted_0,note_indices,uniform_steps_1,labels_1,predicted_1,note_indices_1,uniform_steps_2,labels_2,predicted_2,note_indices_2,uniform_steps_3,labels_3,predicted_3,note_indices_3,accuracy,labels_decoded,preds_decoded
0,/home/ui556004/datasets/rnbert_csvs/MozartPSDa...,"[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 2, 4, 4, ...","[4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 4, 4, ...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, ...","[0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24...","[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 2, 4, 4, ...","[4, 4, 4, 4, 4, 4, 4, 4, 6, 6, 6, 6, 6, 4, 4, ...","[0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 0, 0, ...","[0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24...","[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 2, 4, 4, ...","[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24...","[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 2, 4, 4, ...","[4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 4, 4, ...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, ...","[0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24...",0.789931,"[M_0.0_0.0M__I_I, M_0.0_0.0M__I_I, M_0.0_0.0M_...","[M_0.0_0.0M__I_I, M_0.0_0.0M__I_I, M_0.0_0.0M_..."
1,/home/ui556004/datasets/rnbert_csvs/MozartPSDa...,"[16, 16, 12, 2, 2, 8, 4, 4, 4, 4, 4, 4, 4, 4, ...","[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, ...","[0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 21, 22...","[16, 16, 12, 2, 2, 8, 4, 4, 4, 4, 4, 4, 4, 4, ...","[4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 6, ...","[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, ...","[0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 21, 22...","[16, 16, 12, 2, 2, 8, 4, 4, 4, 4, 4, 4, 4, 4, ...","[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 21, 22...","[16, 16, 12, 2, 2, 8, 4, 4, 4, 4, 4, 4, 4, 4, ...","[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 15, 15, 15, 15,...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 11, 11, 11, 11,...","[0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 21, 22...",0.784346,"[M_0.0_0.0M__I_I, M_0.0_0.0M__I_I, M_0.0_0.0M_...","[M_0.0_0.0M__I_I, M_0.0_0.0M__V_I, M_0.0_0.0M_..."
2,/home/ui556004/datasets/rnbert_csvs/MozartPSDa...,"[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, ...","[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 3, 6, 9, 12, 13, 16, 19, 22, 25, 28, 31, 3...","[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, ...","[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 3, 6, 9, 12, 13, 16, 19, 22, 25, 28, 31, 3...","[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, ...","[13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 1...","[9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, ...","[0, 3, 6, 9, 12, 13, 16, 19, 22, 25, 28, 31, 3...","[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, ...","[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, ...","[0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 0, 0, 0, ...","[0, 3, 6, 9, 12, 13, 16, 19, 22, 25, 28, 31, 3...",0.701157,"[M_0.0_7.0M__I_I, M_0.0_7.0M__I_I, M_0.0_7.0M_...","[M_0.0_7.0M__I_I, M_0.0_7.0M__I_I, M_0.0_7.0M_..."
3,/home/ui556004/datasets/rnbert_csvs/RomeData/S...,"[8, 4, 4, 8, 4, 4, 8, 4, 4, 8, 4, 4, 8, 4, 4, ...","[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, ...","[0, 4, 8, 12, 16, 20, 24, 28, 32, 36, 40, 44, ...","[8, 4, 4, 8, 4, 4, 8, 4, 4, 8, 4, 4, 8, 4, 4, ...","[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, ...","[0, 4, 8, 12, 16, 20, 24, 28, 32, 36, 40, 44, ...","[8, 4, 4, 8, 4, 4, 8, 4, 4, 8, 4, 4, 8, 4, 4, ...","[9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, ...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...","[0, 4, 8, 12, 16, 20, 24, 28, 32, 36, 40, 44, ...","[8, 4, 4, 8, 4, 4, 8, 4, 4, 8, 4, 4, 8, 4, 4, ...","[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4

In [8]:
df[df["accuracy"] < 0.4]

,path,uniform_steps,labels_0,predicted_0,note_indices,uniform_steps_1,labels_1,predicted_1,note_indices_1,uniform_steps_2,labels_2,predicted_2,note_indices_2,uniform_steps_3,labels_3,predicted_3,note_indices_3,accuracy
17,/home/ui556004/datasets/rnbert_csvs/MedtnerTal...,"[4, 2, 2, 2, 2, 4, 4, 2, 2, 2, 2, 4, 2, 2, 4, ...","[4, 4, 4, 4, 4, 10, 4, 4, 5, 5, 5, 4, 4, 4, 6,...","[0, 4, 4, 4, 0, 0, 0, 0, 1, 1, 1, 0, 0, 2, 2, ...","[0, 7, 15, 21, 27, 30, 33, 36, 41, 46, 51, 56,...","[4, 2, 2, 2, 2, 4, 4, 2, 2, 2, 2, 4, 2, 2, 4, ...","[4, 6, 6, 6, 5, 4, 4, 4, 7, 7, 7, 5, 5, 5, 4, ...","[0, 3, 3, 3, 1, 1, 0, 0, 3, 3, 3, 1, 1, 0, 0, ...","[0, 7, 15, 21, 27, 30, 33, 36, 41, 46, 51, 56,...","[4, 2, 2, 2, 2, 4, 4, 2, 2, 2, 2, 4, 2, 2, 4, ...","[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 7, 15, 21, 27, 30, 33, 36, 41, 46, 51, 56,...","[4, 2, 2, 2, 2, 4, 4, 2, 2, 2, 2, 4, 2, 2, 4, ...","[4, 6, 6, 6, 6, 8, 5, 5, 5, 5, 5, 4, 4, 4, 7, ...","[0, 2, 5, 5, 2, 2, 1, 1, 1, 1, 1, 0, 0, 3, 3, ...","[0, 7, 15, 21, 27, 30, 33, 36, 41, 46, 51, 56,...",0.357397
74,/home/ui556004/datasets/rnbert_csvs/GriegLyric...,"[4, 8, 4, 8, 4, 12, 8, 4, 4, 4, 4, 4, 4, 4, 12...","[6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, ...","[0, 0, 0, 0, 0, 2, 2, 2, 0, 0, 0, 2, 2, 2, 2, ...","[0, 1, 2, 3, 4, 5, 9, 12, 13, 14, 15, 16, 17, ...","[4, 8, 4, 8, 4, 12, 8, 4, 4, 4, 4, 4, 4, 4, 12...","[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 2, 3, 4, 5, 9, 12, 13, 14, 15, 16, 17, ...","[4, 8, 4, 8, 4, 12, 8, 4, 4, 4, 4, 4, 4, 4, 12...","[25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 2...","[21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 2...","[0, 1, 2, 3, 4, 5, 9, 12, 13, 14, 15, 16, 17, ...","[4, 8, 4, 8, 4, 12, 8, 4, 4, 4, 4, 4, 4, 4, 12...","[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, ...","[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 2, 3, 4, 5, 9, 12, 13, 14, 15, 16, 17, ...",0.361823
87,/home/ui556004/datasets/rnbert_csvs/BpsFhData/...,"[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, ...","[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, ...","[0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 2, 2, ...","[0, 1, 2, 3, 4, 5, 6, 8, 9, 11, 12, 14, 15, 17...","[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, ...","[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 2, 3, 4, 5, 6, 8, 9, 11, 12, 14, 15, 17...","[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, ...","[13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 1...","[9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, ...","[0, 1, 2, 3, 4, 5, 6, 8, 9, 11, 12, 14, 15, 17...","[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, ...","[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 6, 6, ...","[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 3, 3, ...","[0, 1, 2, 3, 4, 5, 6, 8, 9, 11, 12, 14, 15, 17...",0.376091
88,/home/ui556004/datasets/rnbert_csvs/BpsFhData/...,"[4, 4, 4, 4, 4, 4, 4, 4, 4, 8, 4, 4, 4, 4, 4, ...","[4, 4, 4, 5, 5, 5, 4, 4, 4, 4, 4, 4, 4, 4, 4, ...","[0, 0, 0, 1, 1, 0, 2, 2, 2, 0, 0, 2, 2, 2, 0, ...","[0, 4, 8, 12, 16, 20, 24, 28, 32, 36, 40, 42, ...","[4, 4, 4, 4, 4, 4, 4, 4, 4, 8, 4, 4, 4, 4, 4, ...","[4, 4, 4, 4, 4, 4, 6, 6, 6, 4, 4, 5, 5, 5, 4, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 4, 8, 12, 16, 20, 24, 28, 32, 36, 40, 42, ...","[4, 4, 4, 4, 4, 4, 4, 4, 4, 8, 4, 4, 4, 4, 4, ...","[15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 1...","[6, 6, 6, 11, 11, 11, 11, 11, 11, 11, 11, 11, ...","[0, 4, 8, 12, 16, 20, 24, 28, 32, 36, 40, 42, ...","[4, 4, 4, 4, 4, 4, 4, 4, 4, 8, 4, 4, 4, 4, 4, ...","[5, 5, 5, 5, 5, 5, 4, 4, 4, 5, 5, 6, 6, 6, 4, ...","[0, 0, 5, 1, 1, 0, 1, 1, 1, 1, 0, 4, 4, 4, 2, ...","[0, 4, 8, 12, 16, 20, 24, 28, 32, 36, 40, 42, ...",0.234568
132,/home/ui556004/datasets/rnbert_csvs/HaydnQsDat...,"[8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 16, 8, 8, 8, 8,...","[4, 6, 8, 6, 6, 6, 6, 8, 8, 5, 7, 7, 5, 5, 6, ...","[2, 2, 2, 2, 2, 2, 2, 4, 4, 3, 3, 3, 1, 0, 2, 

In [12]:
df[df["accuracy"] > 0.95]

,path,uniform_steps,labels_0,predicted_0,note_indices,uniform_steps_1,labels_1,predicted_1,note_indices_1,uniform_steps_2,labels_2,predicted_2,note_indices_2,uniform_steps_3,labels_3,predicted_3,note_indices_3,accuracy
10,/home/ui556004/datasets/rnbert_csvs/TextbookDa...,"[8, 2, 2, 4, 8, 8, 32]","[4, 4, 4, 4, 6, 4, 6]","[0, 0, 0, 0, 2, 0, 2]","[0, 4, 8, 12, 16, 20, 24]","[8, 2, 2, 4, 8, 8, 32]","[4, 5, 5, 4, 5, 4, 4]","[0, 1, 1, 0, 1, 0, 0]","[0, 4, 8, 12, 16, 20, 24]","[8, 2, 2, 4, 8, 8, 32]","[5, 16, 16, 16, 21, 21, 21]","[1, 12, 12, 12, 17, 17, 17]","[0, 4, 8, 12, 16, 20, 24]","[8, 2, 2, 4, 8, 8, 32]","[4, 10, 10, 5, 7, 5, 4]","[0, 6, 6, 1, 3, 1, 0]","[0, 4, 8, 12, 16, 20, 24]",1.000000
13,/home/ui556004/datasets/rnbert_csvs/TextbookDa...,"[8, 4, 4, 8, 8, 32]","[6, 4, 4, 6, 4, 6]","[2, 0, 0, 2, 0, 2]","[0, 4, 8, 12, 16, 20]","[8, 4, 4, 8, 8, 32]","[4, 5, 5, 6, 4, 4]","[0, 1, 1, 2, 0, 0]","[0, 4, 8, 12, 16, 20]","[8, 4, 4, 8, 8, 32]","[27, 21, 21, 21, 21, 21]","[23, 17, 17, 17, 17, 17]","[0, 4, 8, 12, 16, 20]","[8, 4, 4, 8, 8, 32]","[4, 7, 7, 4, 5, 4]","[0, 3, 3, 0, 1, 0]","[0, 4, 8, 12, 16, 20]",1.000000
78,/home/ui556004/datasets/rnbert_csvs/TextbookDa...,"[8, 4, 4, 16]","[6, 4, 5, 4]","[2, 0, 1, 0]","[0, 4, 8, 12]","[8, 4, 4, 16]","[4, 4, 4, 4]","[0, 0, 0, 0]","[0, 4, 8, 12]","[8, 4, 4, 16]","[27, 6, 6, 6]","[23, 2, 2, 2]","[0, 4, 8, 12]","[8, 4, 4, 16]","[4, 5, 5, 4]","[0, 1, 1, 0]","[0, 4, 8, 12]",1.000000
81,/home/ui556004/datasets/rnbert_csvs/TextbookDa...,"[8, 8, 8, 8, 32]","[6, 6, 6, 4, 6]","[2, 2, 2, 0, 2]","[0, 4, 8, 11, 15]","[8, 8, 8, 8, 32]","[4, 4, 6, 4, 4]","[0, 0, 2, 0, 0]","[0, 4, 8, 11, 15]","[8, 8, 8, 8, 32]","[27, 19, 19, 19, 19]","[23, 15, 15, 15, 15]","[0, 4, 8, 11, 15]","[8, 8, 8, 8, 32]","[4, 6, 4, 5, 4]","[0, 2, 0, 1, 0]","[0, 4, 8, 11, 15]",1.000000
95,/home/ui556004/datasets/rnbert_csvs/RomeData/R...,"[2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, ...","[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 2, 3, 5, 6, 8, 9, 12, 14, 16, 19, 21, 24, ...","[2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, ...","[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 2, 3, 5, 6, 8, 9, 12, 14, 16, 19, 21, 24, ...","[2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, ...","[13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 1...","[9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, ...","[0, 2, 3, 5, 6, 8, 9, 12, 14, 16, 19, 21, 24, ...","[2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, ...","[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 2, 3, 5, 6, 8, 9, 12, 14, 16, 19, 21, 24, ...",0.972603
110,/home/ui556004/datasets/rnbert_csvs/TextbookDa...,"[8, 8, 8, 8]","[6, 4, 7, 6]","[2, 0, 3, 2]","[0, 4, 8, 12]","[8, 8, 8, 8]","[4, 5, 4, 4]","[0, 1, 0, 0]","[0, 4, 8, 12]","[8, 8, 8, 8]","[27, 23, 23, 23]","[23, 19, 19, 19]","[0, 4, 8, 12]","[8, 8, 8, 8]","[4, 6, 9, 4]","[0, 2, 5, 0]","[0, 4, 8, 12]",1.000000
112,/home/ui556004/datasets/rnbert_csvs/TextbookDa...,"[16, 16, 32]","[4, 5, 4]","[0, 1, 0]","[0, 4, 8]","[16, 16, 32]","[4, 4, 4]","[0, 0, 0]","[0, 4, 8]","[16, 16, 32]","[4, 4, 15]","[0, 0, 11]","[0, 4, 8]","[16, 16, 32]","[4, 16, 4]","[0, 12, 0]","[0, 4, 8]",1.000000
116,/home/ui556004/datasets/rnbert_csvs/BachChoral...,"[4, 4, 4, 4, 8, 4, 4, 16, 4, 4, 4, 4, 4, 4, 4,...","[4, 4, 4, 4, 4, 4, 4, 4, 6, 6, 4, 4, 4, 4, 4, ...","[0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, ...","[0, 4, 8, 12, 16, 20, 24, 28, 32, 36, 40, 44, ...","[4, 4, 4, 4, 8, 4, 4, 16, 4, 4, 4, 4, 4, 4, 4,...","[4, 4, 5, 5, 4, 4, 4, 4, 4, 4, 5, 5, 4, 4, 4, ...","[0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, ...","[0, 4, 8, 12, 16, 20, 24, 28, 32, 36, 40, 44, ...","[4, 4, 4, 4, 8, 4, 4, 16, 4, 4, 4, 4, 4, 4, 4,...","[15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 1...","[11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 1...","[0, 4, 8, 12, 16, 20, 24, 28, 32, 36, 40, 44, ...","[4, 4, 4, 4, 8, 4, 4, 16, 4, 4, 

In [29]:
from music21 import roman, key
rn1 = roman.RomanNumeral('viio42/VI', key.Key('C'))
rn1

<music21.roman.RomanNumeral viio42/VI in C major>

In [17]:
import re
from typing import Optional
from music21 import roman, key

# -------------------------------------------------------------------------
# helpers

_PC_TO_TONIC = {
    0: "C",
    1: "C#",
    2: "D",
    3: "Eb",
    4: "E",
    5: "F",
    6: "F#",
    7: "G",
    8: "Ab",
    9: "A",
    10: "Bb",
    11: "B",
}


def _format_alt_for_music21(alt: str) -> str:
    """
    Map your accidental symbols to something music21 understands.
    - 'x'  -> '##' (double sharp)
    - 'xb' -> 'bb' (double flat)
    """
    if alt == "x":
        return "##"
    if alt == "xb":
        return "bb"
    return alt


def _split_primary_secondary(ps_token: str):
    """
    ps_token examples:
      '_V_I', 'bII_I', '#VII_VI', '_xbVII', 'VIIbVI', '_x_x', etc.

    Returns (primary_str, secondary_str)
    where each is alteration+degree: 'V', 'bII', '#VII', 'xbVII', etc.
    """
    # Strip leading underscore(s) first
    s = ps_token.lstrip("_")
    if not s:
        return None, None

    parts = s.split("_")
    parts = [p for p in parts if p != ""]

    if len(parts) == 2:
        # typical '_V_I', '_V_V', etc.
        return parts[0], parts[1]
    elif len(parts) == 1:
        # packed form like 'VIIbVI', 'V#III', 'xbVII', etc.
        m = re.match(
            r"^(?P<alt1>b|#|x|xb)?(?P<deg1>I{1,3}|IV|V|VI|VII)"
            r"(?P<alt2>b|#|x|xb)?(?P<deg2>I{1,3}|IV|V|VI|VII)?$",
            parts[0],
        )
        if not m:
            # no structured match – treat everything as primary
            return parts[0], None

        alt1 = m.group("alt1") or ""
        deg1 = m.group("deg1")
        alt2 = m.group("alt2") or ""
        deg2 = m.group("deg2")

        primary = alt1 + deg1
        secondary = alt2 + deg2 if deg2 else None
        return primary, secondary
    else:
        return None, None


def _split_alt_degree(deg_token: Optional[str]):
    """
    deg_token like 'V', 'bII', '#VII', 'xVI', 'xbVII', or None.
    Returns (accidental, numeral) e.g. ('b', 'II'), ('', 'V').
    """
    if deg_token is None:
        return None, None

    m = re.match(r"^(?P<alt>b|#|x|xb)?(?P<deg>I{1,3}|IV|V|VI|VII)$", deg_token)
    if not m:
        # something exotic – return "no alt" and the raw token as degree
        return "", deg_token

    alt = m.group("alt") or ""
    deg = m.group("deg")
    return alt, deg


def _build_rn_main(primary_deg_full: str, quality: str, inversion: str) -> str:
    """
    Build the *main* Roman numeral (without /secondary) in music21 style,
    including quality & inversion.

    Special case: quality == 'aug6' → treat as Italian augmented 6th: 'It+6'.
    """
    # ---- special case: augmented sixth chords ---------------------------
    if quality == "aug6":
        # You could make this more sophisticated later (Fr+6/Ger+6),
        # but as a baseline we map *all* aug6 to It+6.
        return "It+6"

    # ---- generic case ---------------------------------------------------
    alt, core = _split_alt_degree(primary_deg_full)
    if alt is None:
        return "N.C."

    alt_out = _format_alt_for_music21(alt)

    is_seventh = quality in {"Mm7", "M7", "m7", "o7", "ø7"}
    is_dim = quality in {"o", "o7"}
    is_half_dim = quality == "ø7"
    is_minor = quality in {"m", "m7"}

    # case (upper vs lower)
    if is_minor or is_dim or is_half_dim:
        core_rn = core.lower()
    else:
        core_rn = core.upper()

    base = alt_out + core_rn

    # add quality markers
    if quality in {"o", "o7"}:
        base += "o"
    elif quality == "ø7":
        base += "ø"
    elif quality == "+":
        base += "+"

    # inversion figures
    if is_seventh:
        inv_map7 = {"0.0": "7", "1.0": "65", "2.0": "43", "3.0": "42"}
        figure = inv_map7.get(inversion, "7")
        rn = base + figure
    else:
        inv_map3 = {"0.0": "", "1.0": "6", "2.0": "64"}
        figure = inv_map3.get(inversion, "")
        rn = base + figure

    return rn


# -------------------------------------------------------------------------
# main function

def chord_string_to_music21(chord_str: str) -> roman.RomanNumeral:
    """
    Convert your RN encoding string, e.g.
      'Mm7_2.0_6.0M__V_I'
      'o7_3.0_0.0M_#VII_VI'
      'aug6_1.0_9.0M__xbVII'
    into a music21 RomanNumeral object.
    """
    # quality, inversion, key_pc_mode, the rest (rn-part)
    q, inv, key_pm, ps = chord_str.split("_", 3)

    # ---- key decoding ---------------------------------------------------
    pc_str, mode_char = key_pm[:-1], key_pm[-1]  # '9.0M' -> '9.0', 'M'
    pc = int(float(pc_str))
    tonic = _PC_TO_TONIC[pc]
    mode = "major" if mode_char == "M" else "minor"
    k = key.Key(tonic, mode)

    # ---- primary/secondary ----------------------------------------------
    primary_str, secondary_str = _split_primary_secondary(ps)

    # for aug6, primary_str might be 'xbVII' but we ignore it in rn symbol
    if primary_str is None:
        rn_main = "N.C."
    else:
        rn_main = _build_rn_main(primary_str, q, inv)

    # attach secondary if present
    if secondary_str:
        # also map its accidental for music21
        alt2, deg2 = _split_alt_degree(secondary_str)
        if deg2 is not None:
            sec = _format_alt_for_music21(alt2) + deg2
            rn_full = f"{rn_main}/{sec}"
        else:
            rn_full = rn_main
    else:
        rn_full = rn_main

    return roman.RomanNumeral(rn_full, k)


In [52]:
all_label_values[-10:]

['Mm7_1.0_0.0M__V_I',
 'M_0.0_0.0M__I_I',
 'm_1.0_0.0M__II_I',
 'm_1.0_0.0M__II_I',
 'm_2.0_9.0m__I_I',
 'm_2.0_9.0m__I_I',
 'm_2.0_9.0m__I_I',
 'M_0.0_9.0m__V_I',
 'M_0.0_9.0m__V_I',
 'M_0.0_9.0m__I_I']

In [51]:
all_predicted_values[-10:]

['Mm7_1.0_0.0M__V_I',
 'M_0.0_0.0M__I_I',
 'm_1.0_9.0m__IV_I',
 'm_1.0_9.0m__IV_I',
 'm_2.0_9.0m__I_I',
 'm_2.0_9.0m__I_I',
 'm_2.0_9.0m__I_I',
 'M_0.0_9.0m__V_I',
 'Mm7_0.0_9.0m__V_I',
 'M_0.0_9.0m__I_I']

In [45]:
for value in list(set(all_predicted_values))[:30]:
    try:
        res = chord_string_to_music21(value)
    except Exception as e:
        print(value)
        print(e)

In [69]:
from music21.analysis import harmonicFunction as hf
from tqdm import tqdm
total_size = len(all_label_values)
not_same = 0
same_function = 0
not_real_rn = 0
for i in tqdm(range(total_size)):
    try:
        true_rn = chord_string_to_music21(all_label_values[i])
        pred_rn = chord_string_to_music21(all_predicted_values[i])
        if true_rn != pred_rn:
            not_same += 1
            hf_true = hf.romanToFunction(true_rn, onlyHauptHarmonicFunction=True)
            hf_pred = hf.romanToFunction(pred_rn, onlyHauptHarmonicFunction=True) 
            if hf_true == hf_pred:
                same_function += 1
    except Exception as e:
        #print(e)
        not_real_rn += 1
        pass
print(f" Total size is {total_size}, wrong answers in total: {not_same}, from them the same function: {round(same_function / not_same *100, 2)}%")
print(f" Skipped {not_real_rn} entries since they could not be converted to rn")

  0%|          | 0/67592 [00:00<?, ?it/s]

100%|██████████| 67592/67592 [03:49<00:00, 294.49it/s]

 Total size is 67592, wrong answers in total: 25626, from them the same function: 53.91%
 Skipped 203 entries since they could not be converted to rn


In [ ]:
import optuna
def create_hyperparams_dict(targets, params):
    hyperparams_dict = {}
    for target in (targets):
        target_params = {}
        # First choose num_linear_layers to use in later loops
        target_params["num_linear_layers"] =  params[f"num_linear_layers_{target}"]
        num_layers = target_params["num_linear_layers"]
        target_params["linear_layers_dim"]  = [
                params[f"layer_dim_{target}_{i}"] for i in range(num_layers)
        ]
        # Activation function per layer
        target_params["activation_fn"] = [ 
             params[f"activation_fn_{target}_{i}"] for i in range(num_layers)
        ]
        # Dropout per layer
        target_params["pooler_dropout"] = [
             params[f"pooler_dropout_{target}_{i}"] for i in range(num_layers)
        ]
        target_params["normalisation"] = [
            params[f"normalisation_{target}_{i}"] for i in range(num_layers)
        ]
        hyperparams_dict[target] = target_params
    return hyperparams_dict

median_pruner = optuna.pruners.MedianPruner(n_warmup_steps=0)
   
sampler = optuna.samplers.TPESampler(seed=0, 
                                        multivariate=True,
                                        warn_independent_sampling=False)
study = optuna.create_study(study_name="nas_layers_extended_new",
                            # in case of 4 classification tasks
                            directions= ["maximize", "maximize","maximize", "maximize"],
                            sampler = sampler,
                            pruner = median_pruner,
                            storage = "sqlite:///optuna_nas.db",
                            load_if_exists=True )


best_trials = study.best_trials
params = best_trials[3].params 

In [ ]:
print("loading the model from hf")
targets =  ["quality", "inversion", "key_pc_mode", "primary_alteration_primary_degree_secondary_alteration_secondary_degree"]
hyperparams_dict = create_hyperparams_dict(targets, params)

path = Path("/hpcwork/ui556004/results/nas_layers_extended_new/trial_0058/checkpoint-34000")
config = BertConfig.from_pretrained(path, force_download=True) # this ensures that the most
                                                                            # up-to-date model is loaded (polina)
config.hyperparams =hyperparams_dict

model = MusicBertMultiTaskTokenClassification.from_pretrained(pretrained_model_name_or_path =path, config=config)

In [ ]:
import h5py
file_path = "/hpcwork/ui556004/results/saved_predictions/test/predictions/primary_alteration_primary_degree_secondary_alteration_secondary_degree.h5"

with h5py.File(file_path, "r") as f:
    # Print all root level object names (aka keys) 
    # these can be group or dataset names 
    print("Keys: %s" % f.keys())
    for i in range(len(f.keys())):
        #print("Keys: %s" % f.keys())
        a_group_key = list(f.keys())[i]

        # get the object type for a_group_key: usually group or dataset
    
        ds_arr = f[a_group_key][()] 
        
        if ds_arr.shape[0] == 3:
            print(ds_arr.shape)

Keys: <KeysViewHDF5 ['logits_0', 'logits_1', 'logits_10', 'logits_100', 'logits_1000', 'logits_1001', 'logits_1002', 'logits_1003', 'logits_1004', 'logits_1005', 'logits_1006', 'logits_1007', 'logits_1008', 'logits_1009', 'logits_101', 'logits_1010', 'logits_1011', 'logits_1012', 'logits_1013', 'logits_1014', 'logits_1015', 'logits_1016', 'logits_1017', 'logits_1018', 'logits_1019', 'logits_102', 'logits_1020', 'logits_1021', 'logits_1022', 'logits_1023', 'logits_1024', 'logits_1025', 'logits_1026', 'logits_1027', 'logits_1028', 'logits_1029', 'logits_103', 'logits_1030', 'logits_1031', 'logits_1032', 'logits_1033', 'logits_1034', 'logits_1035', 'logits_1036', 'logits_1037', 'logits_1038', 'logits_1039', 'logits_104', 'logits_1040', 'logits_1041', 'logits_1042', 'logits_1043', 'logits_1044', 'logits_1045', 'logits_1046', 'logits_1047', 'logits_1048', 'logits_1049', 'logits_105', 'logits_1050', 'logits_1051', 'logits_1052', 'logits_1053', 'logits_1054', 'logits_1055', 'logits_1056', 'lo

In [36]:
content = []
file_path = "/hpcwork/ui556004/results/musicbert_hf/test/outputs/predictions/primary_alteration_primary_degree_secondary_alteration_secondary_degree.txt"
with open(file_path) as f:
    rows = [line.strip().split() for line in f]
len(rows[0])

1876

In [38]:
Counter(rows[0])

Counter({'_I_I': 679,
         '_V_I': 540,
         '_II_I': 270,
         '_IV_I': 123,
         '_VII_I': 80,
         '_III_I': 54,
         '_VI_I': 49,
         '_VII_V': 33,
         '_V_III': 33,
         '_V_V': 15})